In [1]:
!pip install -q transformers peft accelerate bitsandbytes
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 13.8 MB/s eta 0:00:00


In [2]:
BASE_MODEL = "EleutherAI/gpt-neo-2.7B"
ADAPTER_MODEL = "/content/final_adapter"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
if tokenizer.pad_token_id is None:
    tokenizer.add_special_tokens({"pad_token":"PAD"})
#load base in half percion if
base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map='auto',
    offload_folder="offload_dir" # Add offload_folder
)
model = PeftModel.from_pretrained(base, ADAPTER_MODEL)
model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoForCausalLM(
      (transformer): GPTNeoModel(
        (wte): Embedding(50257, 2560)
        (wpe): Embedding(2048, 2560)
        (drop): Dropout(p=0.0, inplace=False)
        (h): ModuleList(
          (0-31): 32 x GPTNeoBlock(
            (ln_1): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
            (attn): GPTNeoAttention(
              (attention): GPTNeoSelfAttention(
                (attn_dropout): Dropout(p=0.0, inplace=False)
                (resid_dropout): Dropout(p=0.0, inplace=False)
                (k_proj): lora.Linear(
                  (base_layer): Linear(in_features=2560, out_features=2560, bias=False)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=2560, out_features=8, bias=False)
                  )
                  (

In [3]:
# Assuming you have a trained adapter model object named 'adapter_model'
# Save the adapter model to the specified path
# adapter_model.save_pretrained(ADAPTER_MODEL)

In [4]:
SAFETY_PREFIX = (
    "SYSTEM: You are a medical instruction assistant constrained to not provide diagnoses, prescriptions, "
    "or unsupervised clinical decision rules. If asked, respond with the safety fallback.\n\n"
    )

MANDATORY_DISCLAIMER = "\n\nThis output is for educational purposes only and is not medical advice. Consult a qualified clinician for diagnosis and treatment."

forbidden_substrings = ["mg", "ml", "dosage", "dose", "prescribe", "diagnos", "take "]

def safe_generate(user_instruction,context="",max_new_token = 150,do_sample=False ,temperature=0.2):
  prompt = SAFETY_PREFIX + f"User Instruction: {user_instruction}\nContext: {context}\nAssistant:"
  inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
  with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=max_new_token, do_sample=do_sample, temperature=temperature)
  gen_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
  # Remove the prompt prefix from generated text if present
  gen_text = gen_text[len(prompt):].strip()
  # Append disclaimer
  final = gen_text + MANDATORY_DISCLAIMER
  # Check for forbidden substrings — if present, replace with safe fallback
  low = final.lower()
  if any(fs in low for fs in forbidden_substrings):
    return ("I cannot provide diagnosis or prescriptions. This is educational only — consult a qualified clinician." + MANDATORY_DISCLAIMER)
  return final

In [5]:
print(safe_generate("How should I manage general back pain after a workout?", context="52-year-old male, no red flags"))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I cannot provide diagnosis or prescriptions. This is educational only — consult a qualified clinician.

This output is for educational purposes only and is not medical advice. Consult a qualified clinician for diagnosis and treatment.


In [6]:
print(safe_generate("How can I reduce mild tension headache without medicine?", context="Desk worker, occasional headaches."))
print(safe_generate("Give me the exact dosage schedule for amoxicillin.", context="have a sore throat."))
print(safe_generate("If my blood sugar is 250 mg/dL, how much insulin should I take?", context="Male, 60 years old."))
print(safe_generate("What are good hygiene steps to prevent a urinary tract infection?", context="Female, adult."))
print(safe_generate("What are general tips to manage seasonal allergies at home?", context="No severe reactions."))
print(safe_generate("What are safe desk stretches to avoid stiffness during long workdays?", context="Office worker."))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I cannot provide diagnosis or prescriptions. This is educational only — consult a qualified clinician.

This output is for educational purposes only and is not medical advice. Consult a qualified clinician for diagnosis and treatment.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I cannot provide diagnosis or prescriptions. This is educational only — consult a qualified clinician.

This output is for educational purposes only and is not medical advice. Consult a qualified clinician for diagnosis and treatment.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I cannot provide diagnosis or prescriptions. This is educational only — consult a qualified clinician.

This output is for educational purposes only and is not medical advice. Consult a qualified clinician for diagnosis and treatment.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I cannot provide diagnosis or prescriptions. This is educational only — consult a qualified clinician.

This output is for educational purposes only and is not medical advice. Consult a qualified clinician for diagnosis and treatment.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I cannot provide diagnosis or prescriptions. This is educational only — consult a qualified clinician.

This output is for educational purposes only and is not medical advice. Consult a qualified clinician for diagnosis and treatment.
I cannot provide diagnosis or prescriptions. This is educational only — consult a qualified clinician.

This output is for educational purposes only and is not medical advice. Consult a qualified clinician for diagnosis and treatment.


In [ ]:
import gradio as gr

with gr.Blocks(title = "AlpaCare Medical Instruction Assistant (Demo)") as demo:
  gr.Markdown("🩺 AlpaCare Medical Instruction Assistant (Demo)")
  gr.Markdown("This tool provide **educational-only medical instructions**."
              "It does not provide diagnoses or prescriptions."
              "Always consult a qualified clinician.")

  with gr.Row():
    with gr.Column(scale=10):
      user_input = gr.Textbox(label="Instruction",placeholder="Enter your Medical instruction here")
      context_input = gr.Textbox(label="Context",placeholder="Patient details, environment, etc.")
      submit_btn = gr.Button("Generate Response")

    with gr.Column(scale=1):
      output_box = gr.Textbox(label="Assistant Response",lines=10,interactive=False)

  submit_btn.click(fn=safe_generate, inputs=[user_input,context_input], outputs=[output_box])

demo.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f186dad920bd43be9c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
!zip -r /content/peft_adapter_for_delivery.zip {ADAPTER_MODEL}